# Vanilla LSTMS, 05052017

val loss 0.49, 

In [2]:
from datetime import datetime
from IPython.display import SVG

import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ProgbarLogger, TensorBoard
from keras_tqdm import TQDMNotebookCallback

from utils import load_embeddings, extract_questions_from_dataframe, save_submission

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
class Config(object):
    VOCABULARY_SIZE = 1193514
    EMBEDDING_DIMENSION = 200
    OFFSET = 3
    OOV_TOKEN = 0  # out of vocabulary
    EOS_TOKEN = 1  # end of sentence
    PAD_TOKEN = 2  # padding to max sentence length
    MAX_SENTENCE_LENGTH = 60
    
    def stamp(self, comment):
        return '{date:%Y%m%d_%H%M}_{comment}'.format(
            date=datetime.now(), comment=comment)

In [4]:
%%time
train_dataframe = pd.read_csv('train.csv')
current_config = Config()

embedding_weights, word2idx = load_embeddings(
    'glove.twitter.27B.200d.txt',
    config=current_config
)

questions_A, questions_B, labels = extract_questions_from_dataframe(
    train_dataframe, 
    config=current_config,
    word2idx=word2idx,
    prediction_mode=False
)

No saved file, preprocessing from scratch


96463 questions preprocessed
CPU times: user 2min 33s, sys: 6.16 s, total: 2min 39s
Wall time: 2min 46s


In [13]:
questions_A.shape

(96463, 60)

In [5]:
shared_lstm_layer = LSTM(
    units=100, 
    return_sequences=False, 
    go_backwards=True, 
    dropout=0.15 + np.random.rand() * 0.25
)
shared_embedding_layer = Embedding(
    input_dim=current_config.VOCABULARY_SIZE + current_config.OFFSET, 
    output_dim=current_config.EMBEDDING_DIMENSION, 
    input_length=current_config.MAX_SENTENCE_LENGTH,
    weights=[embedding_weights],
    trainable=False
)

input_A = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_A = shared_embedding_layer(input_A)
sentence_representation_A = shared_lstm_layer(embeddings_A)
dropout_A = Dropout(0.5)(sentence_representation_A)

input_B = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_B = shared_embedding_layer(input_B)
sentence_representation_B = shared_lstm_layer(embeddings_B)
dropout_B = Dropout(0.5)(sentence_representation_B)

merged_model = concatenate([dropout_A, dropout_B])
predictions = Dense(1, activation='sigmoid')(merged_model)

model = Model(inputs=[input_A, input_B], outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 60, 200)       238703400   input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 100)           120400      embedding_1[0][0]       

In [9]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint(
    monitor='val_loss',
    filepath=current_config.stamp(comment='1') + '.h5', 
    save_best_only=True, 
    save_weights_only=True
)
progress_bar = ProgbarLogger(count_mode='samples')
tensorboard = TensorBoard(
    log_dir='./logs', 
    histogram_freq=1,
)
tqdm_bar = TQDMNotebookCallback()

In [10]:
%%time

training_logs = model.fit(
    x=[questions_A, questions_B], 
    y=labels, 
    epochs=1, 
    batch_size=1024,
    validation_split=0.1, 
    verbose=1,
    callbacks=[early_stopping, model_checkpoint, progress_bar, tensorboard, tqdm_bar]
)

Train on 86816 samples, validate on 9647 samples


INFO:tensorflow:Summary name embedding_1/embeddings:0 is illegal; using embedding_1/embeddings_0 instead.


INFO:tensorflow:Summary name lstm_1/kernel:0 is illegal; using lstm_1/kernel_0 instead.


INFO:tensorflow:Summary name lstm_1/recurrent_kernel:0 is illegal; using lstm_1/recurrent_kernel_0 instead.


INFO:tensorflow:Summary name lstm_1/bias:0 is illegal; using lstm_1/bias_0 instead.


...

86016/86816 [============================>.] - ETA: 5s - loss: 0.5724 - acc: 0.7277 

86016/|/[loss: 0.572, acc: 0.728]  99%|| 86016/86816 [09:30<00:04, 170.24it/s]

86816/86816 [==============================] - 587s - loss: 0.5714 - acc: 0.7284 - val_loss: 0.4926 - val_acc: 0.7926

86816/86816 [==============================] - 617s - loss: 0.5714 - acc: 0.7284 - val_loss: 0.4926 - val_acc: 0.7926



CPU times: user 24min 26s, sys: 5min 42s, total: 30min 9s
Wall time: 10min 28s


In [12]:
model.load_weights('20170504_2334_1.h5')
min(training_logs.history['val_loss'])

0.49261399169060632

In [16]:
test_dataframe = pd.read_csv('test.csv')
test_questions_A, test_questions_B, _ = extract_questions_from_dataframe(
    test_dataframe,
    config=current_config,
    word2idx=word2idx,
    prediction_mode=True
)

No saved file, preprocessing from scratch


2345796 questions preprocessed


In [17]:
predictions = model.predict(
    x=[test_questions_A, test_questions_B], 
    batch_size=8192, 
    verbose=1
)

   8192/2345796 [..............................] - ETA: 8749s

  16384/2345796 [..............................] - ETA: 7783s

  24576/2345796 [..............................] - ETA: 7248s

  32768/2345796 [..............................] - ETA: 6937s

  40960/2345796 [..............................] - ETA: 6804s

...

2318336/2345796 [============================>.] - ETA: 78s 

2326528/2345796 [============================>.] - ETA: 54s

2334720/2345796 [============================>.] - ETA: 31s

2342912/2345796 [============================>.] - ETA: 8s 

2345796/2345796 [==============================] - 6690s  

In [18]:
save_submission(predictions, current_config)

,is_duplicate
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,1
8,0
9,0


In [19]:
!wc -l 20170505_0156_1.csv

 2345797 20170505_0156_1.csv
